# patent가 인용한 application 찾기

### USPTO xml file에서 cited application 정보 추출
- Fung에서 이제 app citation 제공 안 한다고 함...
- 그럼 직접 해야지 뭐!
- 나중에 join하면 되니까 => DB schema 함 짜야겠네

### 추출할 element
patent_id: 첫 doc-number  
app_cited: citation / patcit / document-id / country='US' & len(doc-number) > 8 => **country != 'US'에서 탈출** (다음 doc으로 ㄱㄱ)   
examiner: citation / category

### some issues on the elements
**patent_id**
- 맨 앞이 D로 시작하는 애들 때문에 D없는 애들 앞에 0이 붙음   
- but, 우리가 관심있는 특허는 D로 시작하는 애들 없음  
- **=> int 만들어서 0 떼고 value Error 뜨면 바로 다음 doc으로 넘겨버리자**

**cit id**
- DN이 연도 / 숫자로 구성 (중간에 '/' 무의미)  
- **=> / 를 공백으로 만들자**
  
**examiner**  
- dummy로 변경

In [1]:
#-*- coding: utf-8 -*-
import os
import re
import csv
import sys
import xml.etree.ElementTree as et
import datetime as dt

os.chdir('E:/apps')

In [2]:
def extract_xml_strings(filename):
    """
    Given a string [filename], opens the file and returns a generator
    that yields tuples. A tuple is of format (year, xmldoc string). A tuple
    is returned for every valid XML doc in [filename]
    """
    # search for terminating XML tag
    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    with open(filename, 'r') as f:
        doc = ''  # initialize current XML doc to empty string
        for line in f:
            doc += line
            endtag = endtag_regex.findall(line) if not endtag else endtag   #is there any endtags?
            if not endtag:
                continue      #if there's no end tag, keep the loop going
            terminate = re.compile('^</{0}>'.format(endtag[0]))    #if there's a end tag, then check whether the doc is finished or not
            if terminate.findall(line):     #if there's a terminator
                yield (doc)                 #export doc and reinitialize a doc
                endtag = ''
                doc = ''

In [3]:
date = '20080101'  #initializing date

with open('app_citation.csv', 'wb') as cit:
    writer = csv.writer(cit,quoting=csv.QUOTE_ALL)
    writer.writerow(['patent_id','app_cited','by_examiner'])
    
    while not date=='20130101':
        filename = 'pat_xmls/ipgb'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            root = et.fromstring(doc)
            n = root.iter('doc-number')
            pid = n.next().text

            try:
                pid = int(pid)                          
                for cit in root.iter('citation'):   #using iter because of multi-assignee apps 
                    did = cit[0][0]
                    cited = cit[1].text              
                    try:
                        if did.find('country').text != 'US':  #if foreign cit, go to next doc
                            break 

                        cid = did.find('doc-number').text
                        if len(cid) < 8:
                            continue

                        cid=cid.replace('/','')
                        cit_dum = 1 if cited.split()[-1] == 'examiner' else 0

                        writer.writerow([pid,cid,cit_dum])
                    
                    except AttributeError:         #if other cit, go to next doc
                        break
                        
            except ValueError:      #if pid != int, go to the next doc
                continue    
                             
        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')

__05~07 새로 붙이기__

In [9]:
date = '20050104'  #initializing date

with open('app_citation.csv', 'ab') as cit:
    writer = csv.writer(cit,quoting=csv.QUOTE_ALL)
    
    while not date=='20080101':
        filename = 'pat_xmls/ipgb'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            root = et.fromstring(doc)
            n = root.iter('doc-number')
            pid = n.next().text

            try:
                pid = int(pid)                          
                for cit in root.iter('citation'):   #using iter because of multi-assignee apps 
                    did = cit[0][0]
                    try:
                        cited = cit[1].text              
                        try:
                            if did.find('country').text != 'US':  #if foreign cit, go to next doc
                                break 

                            cid = did.find('doc-number').text
                            if len(cid) < 8:
                                continue

                            cid=cid.replace('/','')
                            cit_dum = 1 if cited.split()[-1] == 'examiner' else 0

                            writer.writerow([pid,cid,cit_dum])

                        except AttributeError:         #if other cit, go to next doc
                            break
                        
                    except IndexError:    #if other cit, go to next doc
                        break
                        
            except ValueError:      #if pid != int, go to the next doc
                continue    
                             
        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')

### examiner or firm 으로 분리

In [2]:
import os
import pandas as pd

os.chdir('E:/apps')
df = pd.read_csv('app_citation.csv')
df.sort_values(by='patent_id',inplace=True)
df.to_csv('app_citation.csv',index=False)
len(df)

7256167

In [4]:
df.sort_values(by='patent_id',inplace=True)
df.iloc[0:10,]

,patent_id,app_cited,by_examiner
6500451,6836907,20020116756,0
6500452,6836913,20020062522,1
6500453,6836913,20030229973,1
6500454,6836914,20020194678,1
6500455,6836916,20030009908,1
6500456,6836917,20020162180,0
6500457,6836917,20040016067,0
6500458,6836918,20020066147,1
6500459,6836918,20030084528,1
6500460,6836921,20040098820,1


In [6]:
df1 = df.loc[df['by_examiner']==0]
len(df1)

4915882

In [7]:
del df1['by_examiner']
df1.to_csv('app_citation_byfirm.csv',index=False)

In [8]:
df1 = df.loc[df['by_examiner']==1]
len(df1)

2340285

In [9]:
del df1['by_examiner']
df1.to_csv('app_citation_byexam.csv',index=False)

In [10]:
4915882.0/7256167

0.6774764142005001